In [1]:
import pandas as pd
import tqdm
import json

In [40]:
# TODO: Download dataset from this notebook instead of from outside it

In [2]:
lines = []
with open("/home/zaristei/repos/musique/raw_data/2wikimultihopqa_train_20k.jsonl") as f:
    for line in f:
        lines.append(json.loads(line))
df_train = pd.DataFrame(lines)
df_train['is_train'] = True
lines = []
with open("/home/zaristei/repos/musique/raw_data/2wikimultihopqa_dev_20k.jsonl") as f:
    for line in f:
        lines.append(json.loads(line))
df_test = pd.DataFrame(lines)
df_test['is_train'] = False
df = pd.concat([df_train, df_test])

In [3]:
df['is_train'].value_counts()

is_train
True     20000
False    12576
Name: count, dtype: int64

In [4]:
df['type'] = df['other_info'].apply(lambda row: row['type'])

In [5]:
df['type'].value_counts()

type
compositional        14339
comparison            9318
bridge_comparison     6834
inference             2085
Name: count, dtype: int64

In [6]:
df['graph'] = df['other_info'].apply(lambda row: row['evidences_id'])

In [7]:
df['graph_size'] = df['graph'].apply(lambda row: len(row))

In [8]:
df['graph_size'].value_counts()

graph_size
2    21390
0     8135
4     2540
5      328
3      183
Name: count, dtype: int64

In [9]:
df = df[df['graph_size'] > 0]

In [10]:
df['is_train'].value_counts()

is_train
True     16377
False     8064
Name: count, dtype: int64

In [11]:
df.columns

Index(['dataset', 'id', 'question_text', 'answer_text', 'answerable',
       'other_info', 'answer_aliases_texts', 'contexts',
       'decomposed_question_text', 'decomposed_instances',
       'component_question_texts', 'is_train', 'type', 'graph', 'graph_size'],
      dtype='object')

In [12]:
refined_df = df[['id', 'question_text', 'answer_text', 'is_train', 'graph', 'type', 'graph_size']]

In [13]:
refined_df.head()
refined_df.to_csv('wikimultihopqa_refined.csv', index=False)

In [65]:
!wget "https://www.dropbox.com/s/lnbhc8yuhit4wm5/wikidata5m_alias.tar.gz?dl=1"

--2024-08-07 16:56:03--  https://www.dropbox.com/s/lnbhc8yuhit4wm5/wikidata5m_alias.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/jfqufhpsryt5sdfzaa63o/wikidata5m_alias.tar.gz?rlkey=hsy3o09yxmi5gpqpc2s4myzvh&dl=1 [following]
--2024-08-07 16:56:03--  https://www.dropbox.com/scl/fi/jfqufhpsryt5sdfzaa63o/wikidata5m_alias.tar.gz?rlkey=hsy3o09yxmi5gpqpc2s4myzvh&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2065c05af264864b246a02c2ce.dl.dropboxusercontent.com/cd/0/inline/CYMQuDkY4mm8tz8BDJERS9ji8JJGNUFMRGZuSv3eQrSPj8ggW9-B__HKsfZoqM94GvbZoacWqN_QxovAJZ7c-EUuHhKPCD352hG7vSIYFBEI-5nwo4dmYbE5Ow1_XjRtsNLPJvygFKV_u-wpjzu6naI1/file?dl=1# [following]
--2024-08-07 16:56:04--  https://uc2065c05af26

In [70]:
!tar -xvf "wikidata5m_alias.tar.gz?dl=1"

wikidata5m_entity.txt
wikidata5m_relation.txt


In [14]:
with open('wikidata5m_relation.txt') as f:
    print(f.readline())

P489	currency symbol description



In [15]:
relation_map = {}
with open('wikidata5m_relation.txt') as f:
    for line in tqdm.tqdm(f):
        parts = line.strip().split('\t')
        for i in range(1, len(parts)):
            if parts[i] not in relation_map:
                relation_map[parts[i]] = []
            relation_map[parts[i]].append(parts[0])

825it [00:00, 193778.40it/s]


In [16]:
# Manually check to see if all of these are valid in WikiData DB, even if they may not be answerable in WikiData5M
for row in refined_df.itertuples():
    for trip in row.graph:
        relation = trip[1]
        if relation not in relation_map:
            print(relation)
        elif len(relation_map[relation]) > 1:
            print(relation, relation_map[relation])

director ['P1037', 'P57']
date of birth
director ['P1037', 'P57']
date of death
director ['P1037', 'P57']
director ['P1037', 'P57']
country ['P3005', 'P17']
country ['P3005', 'P17']
director ['P1037', 'P57']
date of death
director ['P1037', 'P57']
date of death
country ['P3005', 'P17']
country ['P3005', 'P17']
director ['P1037', 'P57']
date of birth
director ['P1037', 'P57']
director ['P1037', 'P57']
director ['P1037', 'P57']
date of birth
country ['P3005', 'P17']
country ['P3005', 'P17']
director ['P1037', 'P57']
director ['P1037', 'P57']
director ['P1037', 'P57']
director ['P1037', 'P57']
country ['P3005', 'P17']
country ['P3005', 'P17']
date of death
country ['P3005', 'P17']
country ['P3005', 'P17']
date of death
date of birth
director ['P1037', 'P57']
date of death
director ['P1037', 'P57']
date of birth
director ['P1037', 'P57']
director ['P1037', 'P57']
date of birth
director ['P1037', 'P57']
director ['P1037', 'P57']
director ['P1037', 'P57']
director ['P1037', 'P57']
director [

In [17]:
entity_set = set()
with open('wikidata5m_entity.txt') as f:
    for line in tqdm.tqdm(f):
        entity_set.add(line.strip().split('\t')[0])

4813491it [00:04, 1109451.61it/s]


In [18]:
missing_entities = set()
missing_entity_idx = set()
for i, row in enumerate(refined_df.itertuples()):
    for trip in row.graph:
        if len(trip) != 3:
            print(trip)
        entities = trip[0], trip[2]
        for entity in entities:
            if entity not in entity_set:
                print(entity)
                missing_entities.add(entity)
                missing_entity_idx.add(i)

Q7069334
date_information
date_information
Q64768110
date_information
date_information
Q30324147
date_information
date_information
date_information
date_information
date_information
date_information
date_information
Q63802341
date_information
date_information
Q3983360
date_information
date_information
Q60738284
date_information
date_information
Q12978262
date_information
date_information
Q3372419
date_information
date_information
date_information
Q59962784
date_information
date_information
Q3998173
date_information
Q28713010
Q331352
Q331352
Q54829004
date_information
Q12830714
date_information
Q60738580
date_information
Q19796493
Q39072549
Q63927168
date_information
date_information
Q1069403
Q51803959
date_information
date_information
Q65939060
date_information
date_information
date_information
Q64174957
date_information
date_information
date_information
date_information
date_information
date_information
date_information
Q18634612
date_information
Q4298967
date_information
date_informa

In [19]:
len(missing_entity_idx)

6851

In [20]:
refined_df.shape

(24441, 7)

In [21]:
# missing relations are ok, but missing entities cannot be mapped to plaintext, so they should be dropped.
refined_df.reset_index(inplace=True, drop=True)
refined_df

,id,question_text,answer_text,is_train,graph,type,graph_size
0,cdbb82ec0baf11ebab90acde48001122,Who is Boraqchin (Wife Of Ögedei)'s father-in-...,Genghis Khan,True,"[[Q7069334, spouse, Q7519], [Q7519, father, Q7...",inference,2
1,228546780bdd11eba7f7acde48001122,What is the date of birth of the director of f...,"July 21, 1926",True,"[[Q83623, director, Q309214], [Q309214, date o...",compositional,2
2,e5150a5a0bda11eba7f7acde48001122,When did the director of film Laughter In Hell...,"August 25, 1963",True,"[[Q6498212, director, Q529568], [Q529568, date...",compositional,2
3,97954d9408b011ebbd84ac1f6bf848b6,Do director of film Coolie No. 1 (1995 Film) a...,no,True,"[[Q2996544, director, Q725970], [Q64768110, di...",bridge_comparison,4
4,13cda43c09b311ebbdb0ac1f6bf848b6,"Are both mountains, Serre Mourene and Monte Ga...",no,True,"[[Q3382389, country, Q29], [Q3861576, country,...",comparison,2
...,...,...,...,...,...,...,...
24436,e6e38bc40bdd11eba7f7acde48001122,Where did Coulson Wallop's father study?,Oxford,False,"[[Q5176173, father, Q6262835], [Q6262835, educ...",compositional,2
24437,35fc19ce0bde11eba7f7acde48001122,What is the place of birth of the director of ...,India,False,"[[Q1637089, director, Q3417929], [Q3417929, pl...",compositional,2
24438,523dfd220bda11eba7f7acde48001122,What is the place of birth of the director of ...,Reykjavík,False,"[[Q818604, director, Q1766199], [Q1766199, pla...",compositional,2
24439,abebf78408c611ebbd90ac1f6bf848b6,Were Massimo Bonanni and Gino Pasqualotto from...,yes,False,"[[Q460758, country of citizenship, Q38], [Q215...",comparison,2


In [22]:
cleaned_df = refined_df.drop(missing_entity_idx)
cleaned_df['is_train'].value_counts()

is_train
True     10969
False     6621
Name: count, dtype: int64

In [25]:
cleaned_df.to_csv('wikimultihopqa_cleaned.csv', index=False)

In [29]:
# Finally, we do the knowledge graph from wikidata5m
!wget "https://www.dropbox.com/s/563omb11cxaqr83/wikidata5m_all_triplet.txt.gz"

--2024-08-07 17:32:18--  https://www.dropbox.com/s/563omb11cxaqr83/wikidata5m_all_triplet.txt.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/6bjl5v1fqocokbpeui29t/wikidata5m_all_triplet.txt.gz?rlkey=wnmo016yogiz3nkl8ljk8kcbv [following]
--2024-08-07 17:32:18--  https://www.dropbox.com/scl/fi/6bjl5v1fqocokbpeui29t/wikidata5m_all_triplet.txt.gz?rlkey=wnmo016yogiz3nkl8ljk8kcbv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7f5d4e5511e66845dfd9483af4.dl.dropboxusercontent.com/cd/0/inline/CYN8GVBH1Zf45p2xk52RBNz9ng3i0Ft1d1TxWAf6B-X7DoxdfhtcVWojbEFgzQYhMW8HbHQQa9nCpbENjihUPdjoMFziDthGVzFdtNwtA_m4wBsxbvfNcJq3GH-vafjEMNNl-UW48NdeJrm_QsSqsTNg/file# [following]
--2024-08-07 17:32:18--  https://uc7f5d4e5511e66

In [30]:
!gzip -d "wikidata5m_all_triplet.txt.gz"

In [31]:
with open('wikidata5m_all_triplet.txt') as f:
    print(f.readline())

Q29387131	P31	Q5



In [32]:
# Substitute entity codes with their aliases
# Picking the first alias for each entity (rather arbitrarily)
alias_map = {}
rel_alias_map = {}
for line in open('wikidata5m_entity.txt'):
    parts = line.strip().split('\t')
    entity_id = parts[0]
    aliases = parts[1:]
    alias_map[entity_id] = aliases[0]
for line in open('wikidata5m_relation.txt'):
    parts = line.strip().split('\t')
    relation_id = parts[0]
    relation_name = parts[1]
    rel_alias_map[relation_id] = relation_name

In [35]:
full_graph = []
missing_total = 0
total = 0
for line in tqdm.tqdm(open('wikidata5m_all_triplet.txt')):
    src, rel, dst = line.strip().split('\t')
    if src not in alias_map:
        missing_total += 1
    if dst not in alias_map:
        missing_total += 1
    if rel not in rel_alias_map:
        missing_total += 1
    total += 3
    full_graph.append([alias_map.get(src, src), rel_alias_map.get(rel, rel), alias_map.get(dst, dst)])
print(f"Missing aliases: {missing_total}/{total}")

21354359it [00:39, 543970.68it/s]

Missing aliases: 15704/64063077


In [36]:
full_graph[:10]

[['lalit kumar goel', 'instance of', 'Huamn'],
 ['ugo riccarelli', 'languages spoken, written or signed', 'italian word'],
 ['Road to Paradise (film)', 'director', 'beaudine, william'],
 ['robert van der horst', 'country of citizenship', 'Reino Hulandes'],
 ['wesley pionteck souza', 'member of sports team', 'Santos F.C.'],
 ['Hezekiah Oladipo Davies', 'instance of', 'Huamn'],
 ['kirakossian', 'country of citizenship', 'Hayastani Hanrapetut’yun'],
 ['mountfair, virginia',
  'located in the administrative territorial entity',
  'albermarle county, virginia'],
 ['juan pascual azorin', 'place of birth', 'yecla, spain'],
 ['The Twilight Zone/Time Enough at Last',
  'followed by',
  'perchance to dream (twilight zone episode)']]

In [39]:
import torch

In [38]:
torch.save(full_graph, 'wikimultihopqa_full_graph.pt')

# Generating Subgraphs

In [41]:
from profiling_utils import create_remote_backend_from_triplets
from rag_feature_store import SentenceTransformerFeatureStore
from rag_graph_store import NeighborSamplingRAGGraphStore
from torch_geometric.loader import RAGQueryLoader
from torch_geometric.nn.nlp import SentenceTransformer
from torch_geometric.datasets.updated_web_qsp_dataset import preprocess_triplet, retrieval_via_pcst
from torch_geometric.data import get_features_for_triplets_groups, Data

/home/zaristei/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer(model_name='sentence-transformers/all-roberta-large-v1').to(device)

In [44]:
fs, gs = create_remote_backend_from_triplets(full_graph, model, model, NeighborSamplingRAGGraphStore, SentenceTransformerFeatureStore, 'encode', 'encode', preprocess_triplet, 'wikidata_graph', node_method_kwargs={"batch_size": 256}).load()